In [44]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import PIL
from PIL import Image
import os
from pathlib import Path
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score

In [2]:
labels = pd.read_csv(r'C:\Users\aaf6\Desktop\WB-2\Data_full\etykiety.csv')
labels = labels['class']

In [3]:
#Wgranie i przygotowanie danych
data = []
names = []
PATH = r'C:\Users\aaf6\Desktop\WB-2\Data_full\dataformodel'
files = Path(PATH).glob('*')
for file in files:
    names.append(str(file).split('\\')[-1])
    image = Image.open(file)
    data.append(np.asarray(image))

In [4]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.01, random_state=42)

In [39]:
train = ImageDataGenerator()
train_dataset = train.flow_from_directory(r"C:\\Users\\aaf6\\Desktop\\WB-2\\Data_full_train\\")

Found 1344 images belonging to 4 classes.


In [40]:
test = ImageDataGenerator()
test_dataset = train.flow_from_directory(r"C:\\Users\\aaf6\\Desktop\\WB-2\\Data_full_test\\")

Found 400 images belonging to 4 classes.


In [ ]:
tf.convert_to_tensor(X_train)

In [ ]:
X_train = K.constant(X_train)

In [31]:
model = keras.Sequential(
    [
        layers.Conv2D(16, (3,3), strides=2, activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(32, (3,3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.1),
        layers.Conv2D(64, (3,3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dropout(0.15),
        layers.Dense(4, activation="softmax")
    ]
)


In [34]:
model_2 = keras.Sequential(
    [
        layers.Conv2D(16, (3,3), strides=2, activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.15),
        layers.Conv2D(32, (3,3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.1),
        layers.Conv2D(64, (3,3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(32, (3,3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(32, activation="relu"),
        layers.Dropout(0.15),
        layers.Dense(4, activation="softmax")
    ]
)


In [36]:
model_3 = keras.Sequential(
    [
        layers.Conv2D(16, (3,3), strides=2, activation="relu"),
        layers.MaxPooling2D(pool_size=(4, 4)),
        layers.Dropout(0.15),
        layers.Conv2D(32, (4,4), activation="relu"),
        layers.MaxPooling2D(pool_size=(4, 4)),
        layers.Dropout(0.1),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dropout(0.15),
        layers.Dense(4, activation="softmax")
    ]
)


In [33]:
model.compile(optimizer="rmsprop",metrics=['accuracy'],loss='categorical_crossentropy')
model.fit(x=train_dataset, batch_size=32, epochs=50)

Epoch 1/50
42/42 [==============================] - 27s 630ms/step - loss: 47.5339 - accuracy: 0.5246
Epoch 2/50
42/42 [==============================] - 26s 625ms/step - loss: 1.8403 - accuracy: 0.6897
Epoch 3/50
42/42 [==============================] - 27s 646ms/step - loss: 1.5047 - accuracy: 0.8021
Epoch 4/50
42/42 [==============================] - 28s 661ms/step - loss: 1.1573 - accuracy: 0.8408
Epoch 5/50
42/42 [==============================] - 27s 651ms/step - loss: 0.7600 - accuracy: 0.8713
Epoch 6/50
42/42 [==============================] - 29s 691ms/step - loss: 0.4510 - accuracy: 0.8951
Epoch 7/50
42/42 [==============================] - 28s 673ms/step - loss: 0.6834 - accuracy: 0.8810
Epoch 8/50
42/42 [==============================] - 28s 669ms/step - loss: 0.2176 - accuracy: 0.9211
Epoch 9/50
42/42 [==============================] - 29s 682ms/step - loss: 0.4008 - accuracy: 0.9003
Epoch 10/50
42/42 [==============================] - 27s 630ms/step - loss: 0.4126 - accur

AttributeError: 'Sequential' object has no attribute 'precict'

In [35]:
model_2.compile(optimizer="rmsprop",metrics=['accuracy'],loss='categorical_crossentropy')
model_2.fit(x=train_dataset, batch_size=32, epochs=20)

Epoch 1/20
42/42 [==============================] - 29s 646ms/step - loss: 11.9510 - accuracy: 0.3065
Epoch 2/20
42/42 [==============================] - 29s 700ms/step - loss: 1.5016 - accuracy: 0.3661
Epoch 3/20
42/42 [==============================] - 29s 686ms/step - loss: 1.2450 - accuracy: 0.4970
Epoch 4/20
42/42 [==============================] - 36s 870ms/step - loss: 0.9306 - accuracy: 0.6295
Epoch 5/20
42/42 [==============================] - 49s 1s/step - loss: 0.7584 - accuracy: 0.7195
Epoch 6/20
42/42 [==============================] - 47s 1s/step - loss: 0.6617 - accuracy: 0.7634
Epoch 7/20
42/42 [==============================] - 48s 1s/step - loss: 0.5751 - accuracy: 0.7894
Epoch 8/20
42/42 [==============================] - 46s 1s/step - loss: 0.4729 - accuracy: 0.8371
Epoch 9/20
42/42 [==============================] - 46s 1s/step - loss: 0.4254 - accuracy: 0.8579
Epoch 10/20
42/42 [==============================] - 46s 1s/step - loss: 0.3795 - accuracy: 0.8594
Epoch 

In [37]:
model_3.compile(optimizer="rmsprop",metrics=['accuracy'],loss='categorical_crossentropy')
model_3.fit(x=train_dataset, batch_size=32, epochs=20)

Epoch 1/20
42/42 [==============================] - 43s 934ms/step - loss: 29.0291 - accuracy: 0.4196
Epoch 2/20
42/42 [==============================] - 38s 904ms/step - loss: 1.4224 - accuracy: 0.5439
Epoch 3/20
42/42 [==============================] - 44s 1s/step - loss: 0.9090 - accuracy: 0.6994
Epoch 4/20
42/42 [==============================] - 42s 1s/step - loss: 0.7423 - accuracy: 0.7634
Epoch 5/20
42/42 [==============================] - 42s 1s/step - loss: 0.5227 - accuracy: 0.8147
Epoch 6/20
42/42 [==============================] - 42s 1s/step - loss: 0.4485 - accuracy: 0.8534
Epoch 7/20
42/42 [==============================] - 42s 993ms/step - loss: 0.3551 - accuracy: 0.8884
Epoch 8/20
42/42 [==============================] - 42s 1s/step - loss: 0.3665 - accuracy: 0.8839
Epoch 9/20
42/42 [==============================] - 43s 1s/step - loss: 0.2924 - accuracy: 0.9085
Epoch 10/20
42/42 [==============================] - 49s 1s/step - loss: 0.2793 - accuracy: 0.9189
Epoch 11/

In [42]:
labels_pred = model.predict(test_dataset)

In [51]:
def chose_max(x):
    return x.index(max(x))

In [55]:
labels_predicted = []
for i in range(labels_pred.shape[0]):
    labels_predicted.append(list(labels_pred[i]).index(max(labels_pred[i])))

In [59]:
np.array(labels_predicted)

array([1, 1, 2, 3, 2, 2, 0, 1, 2, 1, 1, 3, 2, 2, 2, 0, 2, 1, 2, 0, 2, 2,
       2, 1, 3, 1, 1, 1, 1, 2, 3, 2, 2, 1, 0, 2, 1, 3, 2, 3, 3, 1, 3, 3,
       2, 1, 1, 2, 2, 2, 2, 1, 0, 1, 1, 3, 2, 3, 2, 1, 3, 0, 0, 0, 3, 3,
       2, 2, 2, 2, 0, 2, 1, 1, 0, 2, 1, 2, 2, 2, 1, 2, 1, 3, 3, 1, 1, 2,
       2, 3, 1, 2, 2, 2, 2, 2, 2, 3, 2, 2, 1, 2, 3, 3, 2, 3, 1, 2, 0, 3,
       3, 2, 1, 0, 3, 1, 0, 2, 2, 3, 2, 0, 2, 3, 0, 2, 2, 3, 2, 3, 3, 1,
       2, 1, 0, 2, 3, 3, 1, 3, 2, 3, 0, 2, 3, 0, 0, 3, 3, 2, 2, 3, 0, 2,
       3, 2, 0, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 0, 2, 2, 1, 3, 1, 2, 2,
       1, 1, 3, 1, 2, 3, 1, 2, 2, 3, 3, 3, 2, 1, 0, 2, 3, 2, 2, 2, 2, 2,
       3, 2, 2, 2, 3, 2, 2, 2, 2, 1, 1, 3, 3, 3, 0, 3, 2, 2, 1, 0, 2, 2,
       1, 3, 3, 2, 1, 2, 1, 3, 2, 3, 3, 1, 2, 1, 2, 3, 3, 3, 2, 3, 2, 0,
       1, 3, 0, 1, 2, 2, 3, 1, 2, 1, 2, 3, 2, 0, 1, 2, 0, 2, 2, 2, 2, 2,
       2, 1, 2, 2, 3, 2, 1, 2, 1, 0, 2, 3, 2, 1, 2, 3, 0, 2, 1, 2, 2, 2,
       2, 0, 1, 1, 2, 2, 2, 2, 1, 1, 0, 2, 2, 1, 2,

In [43]:
test_dataset.classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3,

In [57]:
accuracy_score(test_dataset.classes, labels_predicted)

0.2575